In [ ]:
import sys
import os
from prefect import task, Flow

abspath = os.path.dirname(os.path.normpath(os.path.abspath(os.path.dirname(''))))

sys.path.append(abspath)

driver_path = os.path.join(abspath, 'drivers', 'chromedriver.exe')

raw_download_path = os.path.normpath(os.path.join(abspath, 'data', 'raw'))

from toolbox.scrapers import *

# Data Structure: ['HCPCS/ CPT/NDC Code', 'Internal Proc Code', 'Other Code',
#        'Procedure Name', 'Charge', 'Medicare Adv - Aetna',
#        'Medicare Adv - BCBS', 'Medicare Adv - Humana', 'Medicare Adv - United',
#        'Medicare Adv - Other', 'Aetna', 'BCBS', 'Cigna', 'Exchange/Ambetter',
#        'Medcost', 'United', 'Other Mgd Care', 'Tricare', 'Workers Comp',
#        'Self Pay', 'De-identified Minimum', 'De-identified Maximum']


In [ ]:
hospital_urls = get_source_urls(driver_path, abspath)


In [ ]:
df_cone = get_cone('cone-health', hospital_urls, raw_download_path)


In [ ]:
df_duke_cdm, df_duke_drg = get_duke('duke-university-hospital', hospital_urls, raw_download_path)


In [ ]:
df_ncb = get_north_carolina_baptist('north-carolina-baptist-hospital', hospital_urls, raw_download_path)


In [ ]:
df_cdm_app, df_drg_app, df_shop_app = get_app('app-regional-health-system', hospital_urls, raw_download_path)


In [36]:
df_catawba = get_catawba('catawba-valley-medical-center', hospital_urls,  raw_download_path)


In [42]:
def curate_catawba(df_catawba: pd.DataFrame) -> pd.DataFrame:
    
  df_catawba['CIGNA HEALTHSPRING_Max_Allowable'] = df_catawba['CIGNA HEALTHSPRING_Max_Allowable'].fillna(df_catawba['CIGNA HEALTH AND LIFE INSURANCE COMPANY_Max_Allowable']).fillna(df_catawba['CIGNA_Max_Allowable'])
  
  df = df_catawba[['HospCode',
   'Code',
   'Description',
   'Code_Type',
   'CMS Shoppable',
   'Package/Line_Level',
   'Min_Allowable_835',
   'Max_Allowable_835',
   'AETNA_Max_Allowable',
   'BCBS OF NC_Max_Allowable',
   'CIGNA HEALTHSPRING_Max_Allowable',
   'HUMANA INC._Max_Allowable',
   'UNITED HEALTHCARE INSURANCE COMPANY_Max_Allowable', 
   'Filename']]

  df.columns = ['Rev Code',
   'DRG/CPT/NDC Code',
   'Description',
   'Code_Type',
   'CMS Shoppable',
   'Package/Line_Level',
   'De-identified Minimum',
   'De-identified Maximum',
   'Aetna',
   'BCBS',
   'Cigna',
   'Humana',
   'UHC',
   'Filename']

  return df

In [43]:
curate_catawba_df = curate_catawba(df_catawba)

In [44]:
curate_catawba_df

,Rev Code,DRG/CPT/NDC Code,Description,Code_Type,CMS Shoppable,Package/Line_Level,De-identified Minimum,De-identified Maximum,Aetna,BCBS,Cigna,Humana,UHC,Filename
0,2019281,00100,Anesthesia for procedure on salivary gland wit...,CPT/HCPCS,NaN,Line,0.0,7452.00,NaN,4516.10,NaN,NaN,0.0,560789196_CatawbaValleyMedicalCenter_StandardC...
1,2019281,00103,Anesthesia for procedure on eyelid,CPT/HCPCS,NaN,Line,0.0,3726.00,0.00,NaN,NaN,NaN,0.0,560789196_CatawbaValleyMedicalCenter_StandardC...
2,2019281,00120,Anesthesia for biopsy of external middle and i...,CPT/HCPCS,NaN,Line,0.0,5842.00,4286.14,2301.47,NaN,0.00,NaN,560789196_CatawbaValleyMedicalCenter_StandardC...
3,2019281,00140,Anesthesia for procedure on eye,CPT/HCPCS,NaN,Line,0.0,5782.56,0.00,1664.56,0.00,0.00,0.0,560789196_CatawbaValleyMedicalCenter_StandardC...
4,2019281,00142,Anesthesia for lens surgery,CPT/HCPCS,NaN,Line,0.0,3839.42,0.00,1564.00,0.00,0.00,0.0,560789196_CatawbaValleyMedicalCenter_StandardC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,2019281,Q9967,"Low osmolar contrast material, 300-399 mg/ml i...",CPT/HCPCS,NaN,Line,0.0,15979.43,772.80,772.80,386.40,0.00,772.8,560789196_CatawbaValleyMedicalCenter_StandardC...
2474,2019281,S0028,"Injection, famotidine, 20 mg",CPT/HCPCS,NaN,Line,0.0,18.17,NaN,NaN,NaN,NaN,NaN,560789196_CatawbaValleyMedicalCenter_StandardC...
2475,2019281,U0002,NaN,CPT/HCPCS,NaN,Line,0.0,5865.00,129.00,129.00,51.31,125.73,5865.0,560789196_CatawbaValleyMedicalCenter_StandardC...
2476,2019281,U0003,NaN,CPT/HCPCS,NaN,Line,0.0,229.00,102.00,229.00,100.00,102.00,229.0,560789196_CatawbaValleyMedicalCenter_StandardC...


In [ ]:
df_comp_cateret, df_desc_cateret = get_cateret('cateret-health-care', hospital_urls,  raw_download_path) #Get's charge master not standard charges


In [ ]:

df_mission = get_mission('mission-health', hospital_urls, raw_download_path) #Get's charge master not standard charges


In [ ]:

df_novant = get_novant('novant-health', hospital_urls, raw_download_path)


In [ ]:
df_nhrmc_ip, df_nhrmc_op = get_nhrmc('nhrmc-health', hospital_urls, raw_download_path)


In [ ]:
df_nhrmc_ip

In [ ]:

df_northern = get_northern('northern-regional', hospital_urls, raw_download_path)


In [ ]:

df_first_moore =  get_first('first-health-moore', hospital_urls, raw_download_path)

df_first_montgomery =  get_first('first-health-montgomery', hospital_urls, raw_download_path)

df_cdm_iredell, df_drg_iredell, df_drg_internet =  get_iredell('iredell-health', hospital_urls, raw_download_path)

df_wakemed_cary = get_wakemed('wakemed-cary', hospital_urls, raw_download_path, driver_path)

df_wakemed_raleigh = get_wakemed('wakemed-raleigh', hospital_urls, raw_download_path, driver_path)